In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **Import and Labeled Data**

In [2]:
import os
import pandas as pd

data = []

#Eğitim verisi
dataset_path = '/content/drive/MyDrive/dataset/datasetauthor'


for author in os.listdir(dataset_path):
    author_path = os.path.join(dataset_path, author)
    if os.path.isdir(author_path):
        for file in os.listdir(author_path):
            file_path = os.path.join(author_path, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                data.append({'text': content, 'author': author})

df = pd.DataFrame(data)

In [3]:
random_data = df.sample(n=20)
random_data.head(20)

,text,author
442,"Türkiye AB’nin, AB Türkiye’nin Neresinde?\n\nB...",HCemal
1112,PKK ‘Suriye Fırsatı’nı Kaçırmayacak!\n\nBu yaz...,RMengi
961,"Futbol Medyası\n\n Zorlu, sinir bozucu, kirli,...",MNHazar
710,İyi Ki Abdi İpekçi Bu Günleri Görmedi!\n\nHabe...,MBaransu
261,İlle De Destansı Olsun\n\nTAYYİP Erdoğan şöyle...,AHakan
241,İmam Hatip Raconu\n\n10-15 yıl önce...\n\n- İ...,AHakan
453,CHP’deki Kafa Karışıklığı Deyince...\n\nEcevit...,HCemal
103,Bir 'Muhaliflere Silah Yardımı' Hikâyesi\n\nDo...,COzdemir
346,Sonra...\n\nHep “sonrayı” bekledik...\n\n“Bira...,BCoskun
87,Samet Söyle Yalan Mı?\n\nYarın basın toplantıs...,COzdemir


# **Text Preprocessing**

Importing Library

In [4]:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import nltk as nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # Download wordnet, which is required by WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Data Cleaning

In [5]:
def clean_text(data):

  # convert letters to lower case
  data["text"]=data["text"].str.lower()

  # remove punctuation and '\n'
  data["text"]=data["text"].replace('[^\w\s]','',regex=True)
  data["text"]=data["text"].replace('\n',' ',regex=True)

  # remove numbers
  data["text"]=data["text"].replace('\d',' ',regex=True)

clean_text(df)
df["text"]

,text
0,müslümanlar islâm tarihi okumalı mı hukuk fak...
1,devlet aklı tecellî etti hocam hocam diyorlar...
2,niçin ille de ben başkan olmalıyım grup topla...
3,düğüncü düğün ve hüzün bu yazının merhûm islâ...
4,naz güzeli bugün ekim cumhuriyet bayramı g...
...,...
1195,barzani sizinle gurur duyuyor gene böyle bi e...
1196,achtung ların başı ekmek parası için alman...
1197,çiçekdağı neşet ertaş kırşehirliydi bi ara y...
1198,alex filan sene evvel iki osmanlı kadırg...


**Stopwords**

In [6]:
def remove_stopwords(data):
  sw = stopwords.words('turkish')
  data['text']=data['text'].apply(lambda x:" ".join(x for x in str(x).split()if x not in sw))

remove_stopwords(df)
df["text"]

,text
0,müslümanlar islâm tarihi okumalı hukuk fakülte...
1,devlet aklı tecellî etti hocam hocam diyorlar ...
2,ille ben başkan olmalıyım grup toplantısı yeri...
3,düğüncü düğün hüzün yazının merhûm islâm âlimi...
4,naz güzeli bugün ekim cumhuriyet bayramı günün...
...,...
1195,barzani sizinle gurur duyuyor gene böyle bi ek...
1196,achtung ların başı ekmek parası almanyaya göçm...
1197,çiçekdağı neşet ertaş kırşehirliydi bi ara yoz...
1198,alex filan sene evvel iki osmanlı kadırgası ak...


# **Data Splitting**

In [7]:
from sklearn.model_selection import train_test_split

X = df['text']
y = df['author']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# **TF-IDF**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

tfidf_uni = TfidfVectorizer(analyzer='word', ngram_range=(1,1), max_features=5000)
X_train_uni = tfidf_uni.fit_transform(X_train)
X_test_uni = tfidf_uni.transform(X_test)




**TF-IDF and Decision Tree**


**TF-IDF + Multi-Layer Perceptron (MLP)**

In [9]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Unigram TF-IDF
tfidf_uni = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), max_features=5000)
X_train_uni = tfidf_uni.fit_transform(X_train)
X_test_uni = tfidf_uni.transform(X_test)

# MLP Model
model_mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
model_mlp.fit(X_train_uni, y_train)
pred_mlp = model_mlp.predict(X_test_uni)

# Sonuç
print("Unigram TF-IDF with MLP Classifier")
print(classification_report(y_test, pred_mlp))


Unigram TF-IDF with MLP Classifier
              precision    recall  f1-score   support

      AAltan       0.89      1.00      0.94         8
AAydintasbas       0.62      0.62      0.62         8
      AHakan       0.89      1.00      0.94         8
 ATuranAlkan       0.70      0.88      0.78         8
    AYArslan       0.89      1.00      0.94         8
     BCoskun       0.70      0.88      0.78         8
     CCandar       0.78      0.88      0.82         8
    COzdemir       0.89      1.00      0.94         8
  DCundioglu       0.80      1.00      0.89         8
  DUAribogan       0.88      0.88      0.88         8
      EArdic       1.00      0.75      0.86         8
      ECakir       0.86      0.75      0.80         8
    GGokturk       0.60      0.38      0.46         8
   HBabaoglu       0.71      0.62      0.67         8
      HCemal       0.78      0.88      0.82         8
       HUluc       0.78      0.88      0.82         8
  IKucukkaya       0.86      0.75      0.80   